In [16]:
import os
import shutil
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter  # log writer to visualize the loss functions
from itertools import product

random_state = 42

---
## Preparation

### Device


In [17]:
# PyTorch Device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("Device: {}".format(device))

Device: cuda


### Set seeds


In [18]:
# For reproducibility, fix all the seeds
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.
    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

Da valutare come vogliamo disporre le directory in base al preprocess che facciamo. Ho lasciato una versione che differenzia in base al preproc per averla già pronta da modificare nel caso serva


In [19]:
# versione Rick
base_dir = "2_NN_outputs/"
model_dir = base_dir + "models/"
runs_dir = base_dir + "runs/"

shutil.rmtree(model_dir, ignore_errors=True)
shutil.rmtree(runs_dir, ignore_errors=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(runs_dir, exist_ok=True)

######################
# if not os.path.exists('results'):
#     os.mkdir('results')

# if not os.path.exists('results/pca'):
#     os.mkdir('results/pca')

# if not os.path.exists('results/no_pca'):
#     os.mkdir('results/no_pca')

### Data Layer


In [20]:
class MyDataset(Dataset):
    # Save X and y as Tensors, accordingly to the type of the data
    # https://pytorch.org/docs/stable/tensors.html
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32, device=device)
        self.y = torch.tensor(y, dtype=torch.float32, device=device)

        # Useful attributes
        self.num_features = X.shape[1]
        self.num_classes = len(np.unique(y))

    # Dataset size
    def __len__(self):
        return self.X.shape[0]

    # Fetch a data sample (single sample or batch) for a given index/es
    # (if the dataset is not in memory, it can read from file system and return the object)
    def __getitem__(self, idx):
        return self.X[idx, :], self.y[idx]

### Defining the neural network architecture


In [21]:
# Neural Network class
# Extend the abstract class "Module"
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()

        # Useful attributes
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Definition of Layers
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)  # input to hidden
        self.fc2 = nn.Linear(self.hidden_size, num_classes)  # hidden to output

        # Activation Function
        self.relu = nn.ReLU()

    # How layers are connected between them
    # This even defines the graph of backpropagation
    def forward(self, x):
        h = self.fc1(x)  # first layer
        h = self.relu(h)  # activation function
        output = self.fc2(h)  # second layer
        return output

    def _get_name(self):
        return "FeedForward"

### Training Function


In [30]:
# Function for the training process
def train_model(
    model: nn.Module,  # instance of class to train
    criterion,  # instance of loss function
    optimizer,  # instance of optimizer
    epochs,  # number of
    train_loader: DataLoader,
    val_loader: DataLoader,
    device,  # to train on
    log_writer,
    log_name,
):
    n_iter = 0
    best_valid_loss = float("inf")  # initialized to worst possible value

    # variabili per early stopping
    ######################################### DA VEDERE
    patience = 10
    epochs_no_improve = 0

    # EPOCHS
    for epoch in range(epochs):
        model.train()  # activate training mode (for BatchNorm or Dropout)

        # BATCHES
        for data, targets in train_loader:  # get_item from MyDataset class (single item or batch)
            data, targets = data.to(device), targets.to(device)  # move data and targets to cpu/gpu

            optimizer.zero_grad()  # gradient to zero

            # Forward pass
            y_pred = model(data)
            display(y_pred)

            # Compute Loss
            loss = criterion(
                y_pred, targets.view(y_pred.shape[0], 1)
            )  # reshape because MSELoss requires same dimensionality
            log_writer.add_scalar("Loss train / batches", loss, n_iter)  # plot the batches

            # Backward pass
            loss.backward()
            optimizer.step()

            n_iter += 1

        # Valuation
        y_test, _, y_pred = test_model(model, val_loader, device)
        loss_val = criterion(y_pred, y_test.view(y_pred.shape[0], 1))
        log_writer.add_scalar("Loss val / epochs", loss_val, epoch)  # plot the epochs

        # Save the model with best loss through the epochs
        if loss_val.item() < best_valid_loss:
            best_valid_loss = loss_val.item()
            torch.save(model.state_dict(), model_dir + log_name)

        # #early stopping
        # #####################################################DA VEDERE
        # else:
        #     epochs_no_improve += 1
        #     if epochs_no_improve == patience:
        #         print('Early stopping!')
        #         break

### Test Function


In [23]:
def test_model(model: nn.Module, data_loader: DataLoader, device) -> tuple[Tensor, Tensor, Tensor]:
    """return:
    - y_test - true lables
    - y_pred_c - has 1 column, where each element is the predicted label with bigger probability among the "c" predicted
    - y_pred - has "c" columns as the number of classes of the test set
    """
    model.eval()  # activate evaluation mode (for BatchNorm or Dropout)

    y_pred = []
    y_test = []

    # ## oppure
    # y_pred = torch.tensor([]).to(device)
    # y_true = torch.tensor([]).to(device)

    for data, targets in data_loader:
        data, targets = data.to(device), targets.to(device)  # move data and targets to cpu/gpu

        y_pred.append(model(data))  # accumulate predictions
        y_test.append(targets)  # accumulate labels

    y_test = torch.stack(y_test).squeeze()  # it's one column (each row is a different sample)
    y_pred = torch.stack(
        y_pred
    ).squeeze()  # there are "c" columns as the number of classes. Each column is the probability (as float number) to that class (each row is a different sample)
    y_pred_c = y_pred.argmax(
        dim=1, keepdim=True
    ).squeeze()  # return max position of prediction array: that is the class I will associate with the sample
    return y_test, y_pred_c, y_pred

---

## Train


### Data Loading


In [24]:
df = pd.read_csv("train.csv")
num_rows, num_cols = df.shape
print("Rows: ", num_rows)
print("Columns: ", num_cols)

Rows:  252175
Columns:  91


In [25]:
print("Null rows:", df.shape[0] - df.dropna().shape[0])
print("Duplicated rows:", df.duplicated().sum())
df.drop_duplicates(inplace=True)

Null rows: 0
Duplicated rows: 52


In [26]:
df_X = df.iloc[:, 1:].values
df_y = df.iloc[:, 0].values
indices = np.arange(df_X.shape[0])  # useful later to split the data in train/val/test

# Separate indices in train/val/set
# "stratify=y" makes sure to keep the classes proportions on the dataset (useful on imbalanced classes)
train_idx, test_idx = train_test_split(indices, test_size=0.2, stratify=df_y, random_state=random_state)
train_idx, val_idx = train_test_split(train_idx, test_size=0.2, stratify=df_y[train_idx], random_state=random_state)

In [27]:
# VEDERE COME PREPROCESSARE
# visto che sono tutte gaussiane
# possiamo normalizzare, batch normalizzare,
# standardizzare + scaler che tolga gli outlier
# altro?? da vedere

In [28]:
## ciclare sotto su batch size per train_loader
# DataLoaders
my_dataset = MyDataset(df_X, df_y)

train_subset = Subset(my_dataset, train_idx)

val_subset = Subset(my_dataset, val_idx)
val_loader = DataLoader(val_subset, batch_size=1)

test_subset = Subset(my_dataset, test_idx)
test_loader = DataLoader(test_subset, batch_size=1)

### Hyperparameters


In [29]:
# valutare hidden_size
hidden_size = [128, 256, 512]  # 128
# oppure
# hidden_sizes =  [256, 512, 1024]

# da valutare batch_size
batch_size = [32, 64]  # 32
# oppure
# batch_size = [8,16,32]

# abbiamo detto che non facciamo drop out
# dropout_p = [0.2, 0.3]  # 0.2

depth = [3, 4, 5]

# da valutare learning_rate
learning_rate = [0.001, 0.1]
# oppure
# learning_rate = [0.01, 0.001]
# learning_rate = [0.1, 0.01, 0.001]

num_epochs = [200]
## o 5 o 10

## capire se è il caso di ciclare anche su quei due parametri commentati
hyperparameters = list(
    product(
        hidden_size,
        depth,
        num_epochs,
        batch_size,
        learning_rate,
        # step_size_lr_decay,
        # momentum
    )
)
n_comb = len(hyperparameters)
print(f"Number of hyperparameter combinations: {n_comb}")

Number of hyperparameter combinations: 36


### Run

Run Tensorboard from the command line:

> tensorboard --logdir nn/runs/


In [32]:
fix_random(random_state)

n_iter = 0
best_loss = np.inf
best_combination = None
best_model = None

## da vedere hyperparametri su cui ciclare
for hidden_size, depth, num_epochs, batch_size, lr in hyperparameters:
    n_iter += 1
    print(f"Iteration {n_iter}/{n_comb}")
    ## da vedere hyperparametri da stampare
    print(f"hidden_size: {hidden_size}, depth: {depth}, batch_size: {batch_size}, lr: {lr}")

    log = f"hidden_size: {hidden_size}, depth: {depth}, batch_size: {batch_size}, lr: {lr}"

    ## if PREPROCESS == TRUE
    ## cambio dir su cui scrivo

    # if pca_t == True:
    #     if os.path.exists('risultati/nn/pca'+log):
    #         print('Model alredy trained. Skipping...')
    #         continue
    #     writer= SummaryWriter('risultati/nn/pca/'+log)
    # else:
    #     if os.path.exists('risultati/nn/no_pca/'+log):
    #         print('Model alredy trained. Skipping...')
    #         continue
    #     writer= SummaryWriter('risultati/nn/no_pca'+log)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

    config = {
        "hidden_size": hidden_size,
        "depth": depth,
        "batch_size": batch_size,
        "lr": lr
        ## altri possibili parametri
    }

    # get model
    model = NeuralNetwork(my_dataset.num_features, hidden_size, my_dataset.num_classes)
    model.to(device)  # move the NN to device

    # FOR REGRESSION TASKS
    # It is mathematically the preferred loss function under the inference framework of maximum likelihood if the distribution of the target variable is Gaussian.
    criterion = nn.MSELoss()
    ## provare anche
    # Mean Absolute Error (MAE) -> less sensitive to outliers
    # Huber Loss -> combination of MAE
    # Log-Cosh Loss -> smooth approximation of the MAE loss function. It is less sensitive to outliers than MSE and more robust than MAE.

    ## QUALE OPTIMIZER USIAMO??
    # optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # Momentum
    # RMSprop

    log_writer = SummaryWriter(runs_dir + model._get_name() + str(n_iter))  # Start tensorboard

    # POTREI FARMI RESTITUIRE IL MIGLIOR MODELLO DAL TRAIN E POI SALVARLO PER USARLO DIRETTAMENTE SOTTO NEL TEST FINALE
    # Train
    train_model(
        model,
        criterion,
        optimizer,
        num_epochs,
        train_loader,
        val_loader,
        device,
        log_writer,
        model._get_name(),
    )

    # Load best model
    model.load_state_dict(torch.load(model_dir + model._get_name()))
    model.to(device)

    # Test after the training
    y_true, y_pred = test_model(model, test_loader, device)
    test_loss = mean_squared_error(y_true, y_pred)
    print("Test loss: {test_loss:.4f} - Best loss: {best_loss:.4f}")

    log_writer.add_hparams(config, {"hparam/test_loss": test_loss})

    if test_loss < best_loss:
        best_loss = test_loss
        best_model = model  ## controllare che vada bene salvarlo così
        best_combination = config

    # Close tensorboard writer after a training
    log_writer.flush()
    log_writer.close()

Iteration 1/36
hidden_size: 128, depth: 3, batch_size: 32, lr: 0.001


tensor([[-111.8737,  -69.8732,  -45.4978,  ...,  -54.0606,  150.7720,
         -101.0543],
        [-121.2442,   15.5505,  -89.5311,  ...,  -28.0234,   18.8668,
         -129.3834],
        [ -66.8996,    6.0704,  -70.4086,  ...,  -17.9268,   18.9979,
         -134.6447],
        ...,
        [ -23.1416,   27.1338,   -0.2235,  ...,   -5.1121,  -19.0287,
          -50.8169],
        [-103.2039,   49.4598,  -36.3058,  ...,   -2.7572,  -43.5037,
          -75.7192],
        [ -57.3192,  -35.5319,   26.3158,  ...,  -33.7568,   -4.0851,
          -46.0595]], device='cuda:0', grad_fn=<AddmmBackward0>)

c:\Users\evang.HOMEEVANGELISTI\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 54])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[ 3.5847e+01,  2.0944e+00,  1.4561e+01,  ...,  7.6139e+01,
          7.9179e+01, -1.1004e+02],
        [ 6.9830e+01,  2.0710e+02, -1.0122e+01,  ...,  1.5144e+02,
         -1.5103e+01, -1.4451e+02],
        [-1.1517e+02, -1.1134e+02,  1.2262e+02,  ...,  8.5003e+01,
          2.1716e+00, -9.7005e+01],
        ...,
        [-5.5936e+01,  4.3823e+01, -6.7995e+01,  ...,  3.3711e+00,
          3.9756e+01, -9.7944e+01],
        [-2.5988e+00,  6.8494e+01,  1.1077e+00,  ...,  6.8042e+01,
          8.0512e+01, -6.5477e+01],
        [-2.9404e+01,  7.9621e-02,  7.7100e+00,  ...,  9.4075e+00,
          3.9694e+01, -3.8329e+01]], device='cuda:0', grad_fn=<AddmmBackward0>)

tensor([[ 2.1763e+01,  8.3546e+01,  6.0662e+01,  ...,  7.8233e+01,
          5.7941e+01, -5.0952e+01],
        [ 2.1351e+00,  6.4560e+01,  3.8036e+01,  ...,  9.3138e+01,
          3.9276e+01, -8.3462e+01],
        [ 6.0166e+00,  4.0978e+01,  9.1957e+00,  ...,  8.7090e-02,
          2.4779e+01, -8.2097e+01],
        ...,
        [-5.1887e+01,  1.5536e+01,  4.2328e+01,  ...,  2.7674e+01,
          5.6119e+01, -1.2367e+02],
        [-1.8811e+01,  5.3549e+01,  3.6896e+01,  ...,  5.1048e+01,
          2.8601e+01, -2.0085e+01],
        [ 3.1652e+01,  1.0555e+01,  2.3438e+01,  ...,  3.8970e+00,
          2.1126e+01, -2.7861e+01]], device='cuda:0', grad_fn=<AddmmBackward0>)

...

tensor([[1241.3346, 1223.3047, 1205.0875,  ..., 1243.1517, 1208.9451,
         1201.5386],
        [1513.1190, 1527.3406, 1514.4152,  ..., 1516.3251, 1488.2252,
         1512.6019],
        [1420.8810, 1379.7793, 1332.8691,  ..., 1307.8578, 1348.4940,
         1394.0646],
        ...,
        [2502.7288, 2625.7229, 2621.5718,  ..., 2569.6519, 2530.5093,
         2557.5591],
        [2122.8669, 2122.6707, 2011.1401,  ..., 2098.5247, 2019.6252,
         2195.9634],
        [1788.0020, 1810.8883, 1809.4052,  ..., 1809.8335, 1743.0791,
         1789.7334]], device='cuda:0', grad_fn=<AddmmBackward0>)

tensor([[2329.1238, 2298.8379, 2251.1104,  ..., 2278.4465, 2306.4736,
         2309.9070],
        [1770.5476, 1736.2609, 1757.1149,  ..., 1761.6515, 1729.8142,
         1773.6807],
        [1019.1781, 1010.9542, 1128.7399,  ..., 1023.1174, 1097.1677,
         1096.4189],
        ...,
        [1557.3020, 1611.9171, 1604.3319,  ..., 1583.0538, 1531.3988,
         1544.3347],
        [1789.1691, 1793.8258, 1766.7141,  ..., 1766.2456, 1741.5303,
         1751.0195],
        [ 511.7371,  544.3398,  504.0370,  ...,  540.2350,  524.9703,
          507.1320]], device='cuda:0', grad_fn=<AddmmBackward0>)

tensor([[1827.1951, 1864.7588, 1860.2299,  ..., 1895.4847, 1815.3961,
         1836.3802],
        [2078.3574, 1946.4460, 2186.0481,  ..., 2134.9016, 2124.5164,
         2149.3047],
        [2405.3362, 2419.8008, 2367.2666,  ..., 2408.9458, 2327.1267,
         2355.5723],
        ...,
        [1728.6132, 1694.1855, 1736.0803,  ..., 1703.7190, 1686.5920,
         1714.5768],
        [1365.3033, 1342.4061, 1277.6013,  ..., 1357.9404, 1403.5663,
         1340.3690],
        [1312.2357, 1310.9163, 1304.7881,  ..., 1262.8281, 1257.7469,
         1271.7046]], device='cuda:0', grad_fn=<AddmmBackward0>)

tensor([[2188.2422, 2213.9683, 2199.0095,  ..., 2172.7305, 2194.0708,
         2201.1704],
        [1623.4763, 1623.8527, 1623.5651,  ..., 1631.1586, 1568.6840,
         1605.4258],
        [1773.5831, 1774.2977, 1723.6819,  ..., 1691.7292, 1808.7180,
         1789.4377],
        ...,
        [2371.6470, 2584.5144, 1997.3778,  ..., 2197.0498, 2355.9023,
         2430.4573],
        [1640.4432, 1670.1447, 1654.1417,  ..., 1647.6531, 1619.0992,
         1636.7532],
        [2237.3071, 2231.2974, 2228.4468,  ..., 2210.0889, 2218.0129,
         2181.5850]], device='cuda:0', grad_fn=<AddmmBackward0>)

KeyboardInterrupt: 

### Best results

In [ ]:
print("Best combination:", best_combination)
print("Best loss:", best_loss)

In [ ]:
y_pred, y_true = test_model(best_model, test_loader, device)

mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print("MSE:", mse)
print("R2:", r2)